In [5]:
import torch
from torchtext import data

SEED = 15618

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = "spacy")
LABEL = data.LabelField(dtype = torch.float)

In [6]:
import pandas as pd
from pandas import DataFrame
import random

In [ ]:
train_file = "Pittsburgh_review.train"
test_file = "Pittsburgh_review.test"

train_data, train_labels, test_data, test_labels = [[] for _ in range(4)]

print(pd.read_csv(train_file))



In [0]:


DEV_CSV = "dev_data.csv"

f_train_data = "dev_text.txt"
f_train_label = "dev_label.txt"
f_test_data = "heldout_text.txt"

dev_data , dev_labels , test_data = [] , [] ,[]

with open(f_train_data,"r",encoding = "utf-8") as fd1 , open(f_train_label,"r",encoding = "utf-8") as fd2 , open(f_test_data,"r",encoding = "utf-8") as fd3:
    for line in fd1:
        dev_data.append(line.strip().replace("<br /><br />"," "))
    for line in fd2:
        dev_labels.append(line.strip())
    for line in fd3:
        # test_data.append([token.text for token in nlp.tokenizer(line.strip().replace("<br /><br />"," "))])
        test_data.append(line.strip().replace("<br /><br />"," "))

DEV_DATA = {"text":dev_data,"label":dev_labels}
TEST_DATA = {"text":test_data}

df_dev = DataFrame(DEV_DATA,columns = ["text","label"])
df_dev.to_csv(DEV_CSV)

df_test = DataFrame(TEST_DATA,columns = ["text"])
df_test.to_csv(TEST_CSV)

dev_set = data.TabularDataset(path = DEV_CSV,format = "csv",fields = [("id",None),("text",TEXT),("label",LABEL)],skip_header = True)
test_data = data.TabularDataset(path = TEST_CSV,format = "csv" , fields = [("id",None),("text",TEXT)],skip_header= True)

train_data , valid_data = dev_set.split()

# # full data set
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

print("Train\t","Len:\t",len(train_data),vars(train_data.examples[0]))
print("Val\t"  ,"Len:\t",len(valid_data),vars(valid_data.examples[0]))
print("Test\t" ,"Len:\t",len(test_data) ,vars(test_data.examples[0]))


Train	 Len:	 17500 {'text': ['Undoubtedly', 'the', 'best', 'heavy', 'metal', 'horror', 'item', 'made', 'in', 'the', 'manically', 'headbangin', "'", '80', "'s", ',', 'which', 'admittedly', 'does', "n't", 'sound', 'like', 'much', 'considering', 'how', 'utterly', 'abysmal', 'many', 'other', 'entries', 'in', 'this', 'odd', 'little', 'fright', 'film', 'sub', '-', 'genre', 'like', '"', 'Hard', 'Rock', 'Zombies', ',', '"', '"', 'Blood', 'Tracks', ',', '"', '"', 'Terror', 'on', 'Tour', ',', '"', 'and', 'the', 'especially', 'ungodly', 'Jon', '-', 'Mikl', 'Thor', '-', 'starring', 'stinker', '"', "Rock'n'Roll", 'Nightmare', '"', 'tended', 'to', 'be', '.', 'That', 'aside', ',', 'this', 'one', 'still', 'deserves', 'props', 'for', 'downplaying', 'the', 'excessive', 'splatter', 'and', 'needlessly', 'flashy', 'special', 'f', '/', 'x', 'razzle', '-', 'dazzle', 'in', 'favor', 'of', 'focusing', 'on', 'adolescent', 'high', 'school', 'characters', 'who', 'are', 'depicted', 'with', 'greater', 'acuity', 'and

In [0]:
MAX_VOCAB_SIZE = 20_000

TEXT.build_vocab(train_data,max_size=MAX_VOCAB_SIZE,vectors="glove.6B.200d", unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print("TEXT:",len(TEXT.vocab),"LABEL:",len(LABEL.vocab))
print(TEXT.vocab.freqs.most_common(20))
print(TEXT.vocab.itos[:10])
print(LABEL.vocab.stoi)

TEXT: 20002 LABEL: 2
[('the', 202946), (',', 191897), ('.', 165203), ('and', 109353), ('a', 109090), ('of', 100593), ('to', 93492), ('is', 76388), ('in', 61239), ('I', 53898), ('it', 53471), ('that', 49216), ('"', 44495), ("'s", 43114), ('this', 42033), ('-', 36763), ('/><br', 35671), ('was', 34954), ('as', 30235), ('with', 29932)]
['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']
defaultdict(<function _default_unk_index at 0x7ff2e59af400>, {'pos': 0, 'neg': 1})


In [0]:
BATCH_SIZE = 256
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator = data.BucketIterator.splits(
        (train_data, valid_data),
        sort_within_batch=True, 
        sort_key=lambda x: len(x.text),
        batch_size=BATCH_SIZE,
        device=device)

In [0]:
import torch.nn as nn

DROP = 0.5
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):       
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)  
        self.rnn = nn.LSTM(embedding_dim,hidden_dim,2,dropout = 0,bidirectional = True)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.dropout = torch.nn.Dropout(DROP)
        
    def forward(self, text):
      
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)
#         res = self.dropout(torch.sum(hidden[0],0))
#         print(output.size())
        res = self.dropout(torch.mean(output,0))
        return self.fc(res)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')


The model has 2,631,241 trainable parameters


In [0]:
import torch.optim as optim
optimizer = torch.optim.Adam(model.parameters())
# optimizer = optim.SGD(model.parameters(), lr=1e-2)

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator: 
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.643 | Train Acc: 62.89%
	 Val. Loss: 0.593 |  Val. Acc: 67.69%
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.486 | Train Acc: 76.90%
	 Val. Loss: 0.431 |  Val. Acc: 81.26%
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 0.356 | Train Acc: 84.89%
	 Val. Loss: 0.369 |  Val. Acc: 84.46%
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.276 | Train Acc: 88.93%
	 Val. Loss: 0.378 |  Val. Acc: 84.63%
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.222 | Train Acc: 91.43%
	 Val. Loss: 0.347 |  Val. Acc: 86.88%
